### Seleção, Cruzamento e Mutação

In [7]:
import random
import numpy as np
import pandas as pd

def gerar_cromossomo():
    cromossomo = []
    
    # Gera posições iniciais (0-29)
    posicoes_iniciais = list(range(30))
    random.shuffle(posicoes_iniciais)
    cromossomo.extend(posicoes_iniciais)
    
    # Gera posições finais (0-29)
    posicoes_finais = list(range(30))
    random.shuffle(posicoes_finais)
    cromossomo.extend(posicoes_finais)
    
    # Gera ordem de movimentação (0-29)
    ordem_movimentacao = list(range(30))
    random.shuffle(ordem_movimentacao)
    cromossomo.extend(ordem_movimentacao)
    
    return cromossomo

def cromossomo_para_dataframe(cromossomo):
    container = list(range(30))
    posicao_inicial = cromossomo[:30]
    posicao_final = cromossomo[30:60]
    ordem_movimentacao = cromossomo[60:90]
    
    df = pd.DataFrame({
        'Container': container,
        'Posição Inicial': posicao_inicial,
        'Posição Final': posicao_final,
        'Ordem de Movimentação': ordem_movimentacao
    })
    
    # Ordenar o DataFrame pela coluna 'Ordem de Movimentação'
    df = df.sort_values(by='Ordem de Movimentação')
    
    return df

def inicializar_populacao(tamanho_populacao):
    populacao = []
    for _ in range(tamanho_populacao):
        cromossomo = gerar_cromossomo()
        populacao.append(cromossomo)
    return populacao

def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

def preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    cal_movimentos = 0
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] - 1
        pos_final = linha['Posição Final'] - 1
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        # Verificar se podemos remover do porto preenchido
        pode_mover = True
        if andar_ini < 1:
            if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                pode_mover = False
        
        
        # Verificar se podemos adicionar à balsa
        if andar_fin > 0:
            if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                pode_mover = False

        #Verfica se o lugar ta ocupado
        if balsa[andar_fin , x_fin, y_fin] == 1:
            pode_mover = False



                
        if pode_mover:
            # Atualizar porto_preenchido e balsa
            porto_preenchido[andar_ini, x_ini, y_ini] = 0
            balsa[andar_fin, x_fin, y_fin] = 1
            cal_movimentos += (
                              ((3-andar_ini)+(3-andar_fin)) #alturas
                              + (abs(x_ini - x_fin))
                              + ((2-y_ini)+(y_fin))
                              )
        else:
            cal_movimentos += 1000
            

    return cal_movimentos

def torneio(populacao, fitness, k=3):
    selecionados = random.sample(list(zip(populacao, fitness)), k)
    selecionados.sort(key=lambda x: x[1])
    return selecionados[0][0]

def pmx_crossover(parent1, parent2, start, end):
    size = end - start + 1
    child1 = parent1[:start] + [-1] * size + parent1[end+1:]
    child2 = parent2[:start] + [-1] * size + parent2[end+1:]
    
    mapping1 = parent1[start:end+1]
    mapping2 = parent2[start:end+1]
    
    # Copy the segment
    child1[start:end+1] = mapping2
    child2[start:end+1] = mapping1
    
    # Resolve conflicts
    for i in range(start):
        if child1[i] in mapping2:
            idx = mapping2.index(child1[i])
            while mapping1[idx] in mapping2:
                idx = mapping2.index(mapping1[idx])
            child1[i] = mapping1[idx]
    
    for i in range(end+1, len(parent1)):
        if child1[i] in mapping2:
            idx = mapping2.index(child1[i])
            while mapping1[idx] in mapping2:
                idx = mapping2.index(mapping1[idx])
            child1[i] = mapping1[idx]
    
    for i in range(start):
        if child2[i] in mapping1:
            idx = mapping1.index(child2[i])
            while mapping2[idx] in mapping1:
                idx = mapping1.index(mapping2[idx])
            child2[i] = mapping2[idx]
    
    for i in range(end+1, len(parent2)):
        if child2[i] in mapping1:
            idx = mapping1.index(child2[i])
            while mapping2[idx] in mapping1:
                idx = mapping1.index(mapping2[idx])
            child2[i] = mapping2[idx]
    
    return child1, child2

def cruzamento(pai, mae):
    filho1, filho2 = pai[:], mae[:]
    
    # Cruzamento para as posições iniciais (0-29)
    ponto_corte1 = random.randint(0, 29)
    ponto_corte2 = random.randint(0, 29)
    start, end = min(ponto_corte1, ponto_corte2), max(ponto_corte1, ponto_corte2)
    filho1[:30], filho2[:30] = pmx_crossover(pai[:30], mae[:30], start, end)
    
    # Cruzamento para as posições finais (30-59)
    ponto_corte1 = random.randint(30, 59)
    ponto_corte2 = random.randint(30, 59)
    start, end = min(ponto_corte1, ponto_corte2) - 30, max(ponto_corte1, ponto_corte2) - 30
    filho1[30:60], filho2[30:60] = pmx_crossover(pai[30:60], mae[30:60], start, end)
    
    # Cruzamento para a ordem de movimentação (60-89)
    ponto_corte1 = random.randint(60, 89)
    ponto_corte2 = random.randint(60, 89)
    start, end = min(ponto_corte1, ponto_corte2) - 60, max(ponto_corte1, ponto_corte2) - 60
    filho1[60:90], filho2[60:90] = pmx_crossover(pai[60:90], mae[60:90], start, end)
    
    return filho1, filho2

def mutacao(cromossomo, taxa_mutacao=0.01):
    if random.random() < taxa_mutacao:
        part = random.randint(0, 2)
        idx1, idx2 = random.sample(range(30), 2)
        if part == 0:
            cromossomo[idx1], cromossomo[idx2] = cromossomo[idx2], cromossomo[idx1]
        elif part == 1:
            cromossomo[30+idx1], cromossomo[30+idx2] = cromossomo[30+idx2], cromossomo[30+idx1]
        else:
            cromossomo[60+idx1], cromossomo[60+idx2] = cromossomo[60+idx2], cromossomo[60+idx1]
    return cromossomo

def algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao=0.01):
    populacao = inicializar_populacao(tamanho_populacao)
    fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
    
    for geracao in range(geracoes):
        nova_populacao = []
        while len(nova_populacao) < tamanho_populacao:
            pai = torneio(populacao, fitness)
            mae = torneio(populacao, fitness)
            
            filho1, filho2 = cruzamento(pai, mae)
            filho1 = mutacao(filho1, taxa_mutacao)
            filho2 = mutacao(filho2, taxa_mutacao)
            
            nova_populacao.append(filho1)
            nova_populacao.append(filho2)
        
        populacao = nova_populacao[:tamanho_populacao]
        fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
        
        melhor_fit = min(fitness)
        print(f"Geração {geracao+1}: Melhor fitness = {melhor_fit}")
    
    melhor_cromossomo = populacao[fitness.index(min(fitness))]
    return melhor_cromossomo, min(fitness)

# Configurações
tamanho_populacao = 200
geracoes = 400
taxa_mutacao = 0.08

melhor_cromossomo, melhor_fit = algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao)
print(f"Melhor cromossomo encontrado: {melhor_cromossomo}")
print(f"Melhor fitness: {melhor_fit}")


Geração 1: Melhor fitness = 13145
Geração 2: Melhor fitness = 10166
Geração 3: Melhor fitness = 11154
Geração 4: Melhor fitness = 12164
Geração 5: Melhor fitness = 11163
Geração 6: Melhor fitness = 12149
Geração 7: Melhor fitness = 12158
Geração 8: Melhor fitness = 11157
Geração 9: Melhor fitness = 10168
Geração 10: Melhor fitness = 10167
Geração 11: Melhor fitness = 13140
Geração 12: Melhor fitness = 13149
Geração 13: Melhor fitness = 11151
Geração 14: Melhor fitness = 8190
Geração 15: Melhor fitness = 13138
Geração 16: Melhor fitness = 12148
Geração 17: Melhor fitness = 13144
Geração 18: Melhor fitness = 11168
Geração 19: Melhor fitness = 10168
Geração 20: Melhor fitness = 13137
Geração 21: Melhor fitness = 11178
Geração 22: Melhor fitness = 13139
Geração 23: Melhor fitness = 11165
Geração 24: Melhor fitness = 10174
Geração 25: Melhor fitness = 10170
Geração 26: Melhor fitness = 12155
Geração 27: Melhor fitness = 11152
Geração 28: Melhor fitness = 10171
Geração 29: Melhor fitness = 9

#### Verifica se ha numeros repetidos


In [8]:
def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

if verifica_repetidos(melhor_cromossomo):
    print("Valores repetidos encontrados!")
else:
    print("Não há valores repetidos.")

Não há valores repetidos.


### Simular Melhor cromomossomo

In [10]:
def Simular_preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []
    

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] - 1
        pos_final = linha['Posição Final'] - 1
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        # Verificar se podemos remover do porto preenchido
        pode_mover = True
        if andar_ini < 1:
            if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                pode_mover = False
        
        # Verificar se podemos adicionar à balsa
        if andar_fin > 0:
            if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                pode_mover = False
        
        if pode_mover:
            # Atualizar porto_preenchido e balsa
            porto_preenchido[andar_ini, x_ini, y_ini] = 0
            balsa[andar_fin, x_fin, y_fin] = 1
            
            print(f"Movendo container {container} de ({a_pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({andar_fin}, {x_fin}, {y_fin})")
            print("Estado atual do Porto Preenchido:")
            print(porto_preenchido)
            print("Estado atual da Balsa:")
            print(balsa)
            print("\n")
        else:
            print(f"não moveu container {container} de ({a_pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({andar_fin}, {x_fin}, {y_fin})")


Simular_preencher_porto(cromossomo_para_dataframe(melhor_cromossomo))

Movendo container 13 de (24)(1, 2, 2) para (10)(0, 3, 0)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]
  [1 1 0]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  [1 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 14 de (18)(1, 0, 2) para (1)(0, 0, 0)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 0]
  [1 1 1]
  [1 1 0]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[1 0 0]
  [0 0 0]
  [0 0 0]
  [1 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 11 de (16)(1, 0, 0) para (3)(0, 0, 2)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[0 1 0]
  [1 1 1]
  [1 1 0]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[1 0 1]
  [0 0 0]
  [0 0 0]
  [1 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0